# **Применение градиентного бустинга с использованием CatBoost**

Градиентный бустинг - метод, который находит оптимальное решение на основе решающих дерьевьев, соединенных в ансамбль. Применяется в задачах, в которых исходные данные неоднородны (разного типа: пол, возраст, вес, уровень зарплаты и пр.)

Градиентный бустинг пытается добавить каждое следующее дерево к насамблю таким образом, чтобы минимизировать функцию потерь и приблизить предсказание к правильному ответу. Отличительная особенность CatBoost - все деревья симметричны



In [2]:
!pip install catboost

     |████████████████████████████████| 67.4 MB 29 kB/s 


In [3]:
import os
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)

import catboost
print(catboost.__version__)

0.26.1


In [4]:
from catboost.datasets import msrank_10k

In [5]:
# Этот датасет от Майкрософт содержит 135 числовых фичей. Мы будем решать задачу регресии, предсказывая занчение нулевого столбца
train_df, test_df = msrank_10k()
train_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137
0,2.0,1,3,3,0,0,3,1.0,1.0,0.000000,0.0,1.0,156,4,0,7,167,6.931275,22.076928,19.673353,22.255383,6.926551,3,3,0,0,6,1,1,0,0,2,1,1,0,0,2,1.000000,1.0,0.000000,...,1,0,0,1,1.000000,1.000000,0.000000,0.000000,1.000000,12.941469,20.592760,0.000000,0.000000,16.766961,-18.567793,-7.760072,-20.838749,-25.436074,-14.518523,-21.710022,-21.339609,-24.497864,-27.690319,-20.203779,-15.449379,-4.474452,-23.634899,-28.119826,-13.581932,3,62,11089534,2,116,64034,13,3,0,0,0.0
1,2.0,1,3,0,3,0,3,1.0,0.0,1.000000,0.0,1.0,406,0,5,5,416,6.931275,22.076928,19.673353,22.255383,6.926551,28,0,3,0,31,8,0,1,0,9,10,0,1,0,11,9.333333,0.0,1.000000,...,0,1,0,1,0.994425,0.000000,1.000000,0.000000,0.995455,20.885118,0.000000,24.233365,0.000000,21.161666,-11.555850,-21.242171,-8.429024,-25.436074,-11.297811,-16.487275,-24.805464,-21.461317,-27.690319,-16.208808,-11.646141,-24.041386,-5.143860,-28.119826,-11.411068,2,54,11089534,2,124,64034,1,2,0,0,0.0
2,0.0,1,3,0,2,0,3,1.0,0.0,0.666667,0.0,1.0,146,0,3,7,156,6.931275,22.076928,19.673353,22.255383,6.926551,14,0,2,0,16,1,0,0,0,1,7,0,1,0,8,4.666667,0.0,0.666667,...,0,0,0,1,0.851903,0.000000,0.720414,0.000000,0.842789,18.140878,0.000000,17.748073,0.000000,18.279205,-12.609065,-21.242171,-14.935056,-25.436074,-12.487989,-18.832941,-24.805464,-23.925663,-27.690319,-18.589543,-11.525277,-24.041386,-14.689844,-28.119826,-11.436378,3,45,3,1,124,3344,14,67,0,0,0.0
3,2.0,1,3,0,3,0,3,1.0,0.0,1.000000,0.0,1.0,287,1,4,7,299,6.931275,22.076928,19.673353,22.255383,6.926551,7,0,3,0,10,2,0,1,0,3,3,0,1,0,4,2.333333,0.0,1.000000,...,0,1,0,1,0.989585,0.000000,1.000000,0.000000,0.995185,15.572998,0.000000,26.759999,0.000000,17.531630,-15.555640,-21.242171,-7.761830,-25.436074,-14.198901,-20.103511,-24.805464,-21.459820,-27.690319,-19.180736,-14.798285,-24.041386,-4.474536,-28.119826,-13.825417,3,56,11089534,13,123,63933,1,3,0,0,0.0
4,1.0,1,3,0,3,0,3,1.0,0.0,1.000000,0.0,1.0,2009,2,4,7,2022,6.931275,22.076928,19.673353,22.255383,6.926551,8,0,3,0,11,2,0,1,0,3,3,0,1,0,4,2.666667,0.0,1.000000,...,0,1,0,1,0.980551,0.000000,1.000000,0.000000,0.989938,7.802556,0.000000,26.759999,0.000000,9.749707,-20.673887,-21.242171,-7.761830,-25.436074,-19.469471,-21.419394,-24.805464,-21.459820,-27.690319,-20.589940,-20.168345,-24.041386,-4.474536,-28.119826,-19.226044,3,64,5,7,256,49697,1,13,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,1291,2,0,1,0,2,1.0,0.0,0.500000,0.0,1.0,1226,0,2,4,1232,1.938186,11.202671,6.180345,10.166706,1.933500,150,0,1,0,151,28,0,0,0,28,122,0,1,0,123,75.000000,0.0,0.500000,...,0,0,0,1,0.944744,0.000000,0.911838,0.000000,0.944564,6.509656,0.000000,7.495232,0.000000,6.499751,-6.103003,-10.721004,-6.204031,-11.685236,-6.103333,-7.902061,-12.793498,-9.857750,-13.942820,-7.888291,-6.289006,-11.023170,-7.061288,-12.482012,-6.289860,2,22,131,2,24231,4897,95,228,0,0,0.0
9996,0.0,1291,2,0,1,1,2,1.0,0.0,0.500000,0.5,1.0,283,0,3,7,293,1.938186,11.202671,6.180345,10.166706,1.933500,32,0,2,1,35,3,0,0,0,3,29,0,2,1,32,16.000000,0.0,1.000000,...,0,0,0,1,0.923653,0.000000,0.911838,0.663331,0.922042,6.258489,0.000000,9.237657,5.337635,6.262674,-7.035700,-10.721004,-5.556660,-10.999016,-6.995962,-10.231807,-12.793498,-9.691863,-13.936850,-10.100825,-7.018727,-11.023170,-6.773731,-11.863726,-6.987696,4,39,0,1,52316,55891,79,124,0,0,0.0
9997,2.0,1291,2,0,2,0,2,1.0,0.0,1.000000,0.0,1.0,1108,0,18,4,1130,1.938186,11.202671,6.180345,10.166706,1.933500,72,0,5,0,77,4,0,2,0,6,68,0,3,0,71,36.0

# Предобработка данных

In [6]:
y = train_df[0]
X = train_df.drop([0, 1], axis=1)

В CatBoost есть класс Pool, который располагает фичи в удобном для обучения порядке. Он может принимать Numpy массивы, Pandas датафреймы а также считывать данные с внешних файлов. Попробуем считать данные с внешнего файла.

In [7]:
dataset_dir = './msrank_10k'

if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

train_df.to_csv(
    os.path.join(dataset_dir, 'train.csv'),
    index=False, sep=',', header=True
)

test_df.to_csv(
    os.path.join(dataset_dir, 'test.csv'),
    index=False, sep=',', header=True
)

Помимо этого нам нужно создать column description file с описанием каждой из колонок. Первый столбец содержит целевую переменную, второй - ID группы, поэтому займемся остальными

In [8]:
from catboost.utils import create_cd

feature_names = dict(map(lambda i: (i, 'Feature ' + str(i)), range(train_df.shape[1] - 2)))

create_cd(
    label=None,
    feature_names=feature_names,
    auxiliary_columns=[1],
    output_path=os.path.join(dataset_dir, 'train.cd')
)

In [9]:
from catboost import Pool

pool = Pool(
    data = '/content/msrank_10k/train.csv',
    delimiter = ',',
    column_description = '/content/msrank_10k/train.cd',
    has_header=True,
)

print(f'Dataset shape: {pool.shape}')

Dataset shape: (10000, 137)


Разделим датасет на обучающую и валидационную выборку

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=42)

In [11]:
train_pool = Pool(data=X_train, label=y_train)
val_pool = Pool(data=X_val, label=y_val)

Фича, которая недавно появилась в CatBoost - **Квантизация** датасета. Квантизация - это уменьшение уникальных значений числовых фичей. Если есть большой датасет, в котором некая независимая переменная для каждого лейбла имеет много уникальных зачений, определенный отрезок из таких значений схлопывается в одно.

Зачем это нужно? Когда есть большое количество уникальных значений у фичи, то для создания решающего правила по ней (больше или меньше какой-то величины) придется перебирать все значения этой фичи. Таким образом, возрастает переобучение модели.

Квантизовать датасет желательно до начала обучения 



In [12]:
train_pool.quantize(border_count=254)

# параметры квантизации должны совпадать для обучения и для валидации
# поэтому сохраняем их в отдельный файл
train_pool.save_quantization_borders('borders.tsv')

# и передаем валидационному пулу
val_pool.quantize(input_borders='borders.tsv')

# Обучение модели

In [59]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(
    iterations=500,
    custom_metric='R2',
    use_best_model=True
)

model.fit(
    train_pool,
    eval_set=val_pool,
    verbose=True
);

Learning rate set to 0.105127
0:	learn: 0.7881564	test: 0.8161793	best: 0.8161793 (0)	total: 13.2ms	remaining: 6.58s
1:	learn: 0.7802071	test: 0.8106293	best: 0.8106293 (1)	total: 26ms	remaining: 6.47s
2:	learn: 0.7726579	test: 0.8055513	best: 0.8055513 (2)	total: 39.5ms	remaining: 6.54s
3:	learn: 0.7655959	test: 0.8000403	best: 0.8000403 (3)	total: 52.1ms	remaining: 6.46s
4:	learn: 0.7608514	test: 0.7973031	best: 0.7973031 (4)	total: 65.4ms	remaining: 6.48s
5:	learn: 0.7552817	test: 0.7926342	best: 0.7926342 (5)	total: 78.7ms	remaining: 6.48s
6:	learn: 0.7509935	test: 0.7896504	best: 0.7896504 (6)	total: 91ms	remaining: 6.41s
7:	learn: 0.7465772	test: 0.7863547	best: 0.7863547 (7)	total: 103ms	remaining: 6.34s
8:	learn: 0.7428570	test: 0.7835261	best: 0.7835261 (8)	total: 117ms	remaining: 6.36s
9:	learn: 0.7400727	test: 0.7817575	best: 0.7817575 (9)	total: 130ms	remaining: 6.35s
10:	learn: 0.7370536	test: 0.7793634	best: 0.7793634 (10)	total: 142ms	remaining: 6.32s
11:	learn: 0.734280

Результат **0.74** не плох для начала. Посмотрим, как можно его улучшить.

## **Feature importances**

Некоторые переменные вносят большой вклад в целевую переменную, некоторые - меньше, а некоторые не вносят вовсе. Давайте попробуем упорядочить наши независимые переменные по степени влияния на предсказание

С указанным флагом prettified=True метод get_feature_importance упорядочивает в порядке убывания признаки так, что они в сумме дают 100 по столбцу Importances

In [18]:
feature_importances = model.get_feature_importance(prettified=True)
feature_importances

,Feature Id,Importances
0,9,37.875365
1,54,6.975665
2,107,4.983286
3,135,4.153035
4,109,4.050824
...,...,...
131,13,0.009646
132,68,0.007591
133,43,0.007101
134,34,0.006882


Попробуем установить границу значимости для нашего набора фичей

In [60]:
low_limit = 0.3
feature_importances[feature_importances['Importances']<low_limit]

,Feature Id,Importances
31,52,0.291360
32,123,0.276029
33,61,0.272828
34,18,0.270750
35,129,0.270351
...,...,...
131,13,0.009646
132,68,0.007591
133,43,0.007101
134,34,0.006882


In [61]:
features_to_drop = feature_importances[feature_importances['Importances']<low_limit]['Feature Id']
y = train_df[0]
X = train_df.drop([0, 1], axis=1)

features_to_drop = np.array(features_to_drop)

for feature in features_to_drop:
  X = X.drop(int(feature), axis=1)

Проделаем все то же самое с разделения выборок

In [62]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=42)

train_pool = Pool(data=X_train, label=y_train)
val_pool = Pool(data=X_val, label=y_val)

In [63]:
train_pool.quantize(border_count=254)

# параметры квантизации должны совпадать для обучения и для валидации
# поэтому сохраняем их в отдельный файл
train_pool.save_quantization_borders('borders_new.tsv')

# и передаем валидационному пулу
val_pool.quantize(input_borders='borders_new.tsv')

In [64]:
model = CatBoostRegressor(
    iterations=4000,
    learning_rate=0.000001,
    custom_metric='R2',
    use_best_model=True
)

model.fit(
    train_pool,
    eval_set=val_pool,
    verbose=True
);

0:	learn: 0.7978476	test: 0.8243779	best: 0.8243779 (0)	total: 19.3ms	remaining: 1m 17s
1:	learn: 0.7978475	test: 0.8243778	best: 0.8243778 (1)	total: 30ms	remaining: 59.9s
2:	learn: 0.7978474	test: 0.8243777	best: 0.8243777 (2)	total: 38.4ms	remaining: 51.1s
3:	learn: 0.7978473	test: 0.8243777	best: 0.8243777 (3)	total: 47.1ms	remaining: 47.1s
4:	learn: 0.7978472	test: 0.8243776	best: 0.8243776 (4)	total: 55.4ms	remaining: 44.3s
5:	learn: 0.7978471	test: 0.8243775	best: 0.8243775 (5)	total: 63.9ms	remaining: 42.5s
6:	learn: 0.7978470	test: 0.8243774	best: 0.8243774 (6)	total: 72.2ms	remaining: 41.2s
7:	learn: 0.7978469	test: 0.8243774	best: 0.8243774 (7)	total: 80.8ms	remaining: 40.3s
8:	learn: 0.7978468	test: 0.8243773	best: 0.8243773 (8)	total: 89.8ms	remaining: 39.8s
9:	learn: 0.7978467	test: 0.8243772	best: 0.8243772 (9)	total: 101ms	remaining: 40.4s
10:	learn: 0.7978466	test: 0.8243771	best: 0.8243771 (10)	total: 110ms	remaining: 39.8s
11:	learn: 0.7978465	test: 0.8243770	best: 0

Результат **0.82** гораздо лучше